# Introduction


Calculating benchmarks results for a bigger model (Qwen2.5-1.5B) before and after adding the new tokens

Initial imports and variable initialization

In [2]:
import torch
import sys
import pathlib
import tqdm
import os
import re
os.chdir('/home/yali/MEGA/Hack The Tockenizer/tests')
sys.path.insert(1, pathlib.Path('..').resolve().as_posix())
from src import utils, loader, hack
from src.DatasetClass import ListDataset, TextDataset
from torch.utils.data import DataLoader


DEVICE                  = 'cuda'
GENERATION_BATCH_SIZE   = 8
MODEL                   = 'Qwen/Qwen2.5-1.5B-Instruct'

Loading the model

In [3]:
model, tokenizer = loader.load_model_and_tokenizer(
    model_name=MODEL,
    device=DEVICE,
    model_kwargs = { 'torch_dtype': torch.bfloat16},
    tokenizer_kwargs={'padding_side': 'left'}
)
original_tokenizer = loader.load_model_and_tokenizer(
    model_name=MODEL,
    device='cpu',
    model_kwargs = { 'torch_dtype': torch.bfloat16},
    tokenizer_kwargs={'padding_side': 'left'}
)[1]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


# CalamePT Benchmark

Considering **ONLY** CalamePT, currently not considering the SuperGluePTPT

In [4]:
import src.benchmark as Benchmark
from src.benchmark.CalamePT import CalamePT

# Removing "SuperGluePTPT" from the Benchmarks
benchmark = Benchmark.Benchmarks([CalamePT()])

# Adding the Batch Size (to generate in parallel)
benchmark.config['parallel_batch_size'] = GENERATION_BATCH_SIZE
benchmark.config['max_new_tokens']      = max(len(tokenizer.encode(x)) for x in CalamePT().df['last_word'].values) + 1   # Maximum tokenization of predicted words

## Base Model

In [5]:
benchmark_results = benchmark.run(model, tokenizer)
print(f"`CalamePT` Accuracy for Baseline Model `{model.name_or_path}` = {benchmark_results['CalamePT']['result']:.2%}")

<Qwen/Qwen2.5-1.5B-Instruct> Calculating inferences for inputs: 100%|██████████| 260/260 [01:18<00:00,  3.31it/s]

`CalamePT` Accuracy for Baseline Model `Qwen/Qwen2.5-1.5B-Instruct` = 45.81%


## Model with additional Tokens

### Adding new tokens to model

1. Fetching the new tokens

In [4]:
# ----------------------------------------------------
#           Train Tokenizer with PT Dataset               
# ----------------------------------------------------
# Step 1. Train a new portuguese vocabulary
pt_tokenizer = hack.TokenizerHack(device=DEVICE).train_tokenizer(trainer_kwargs={'vocab_size': 10000})

# Step 2. Find tokens in `pt_tokenizer` not in 
new_tokens = set(pt_tokenizer.get_vocab().keys())
new_tokens = new_tokens.difference(set(tokenizer.vocab.keys()))
# Removing the 'Ġ' tokens and fixing maybe some others
new_tokens = set([tokenizer.decoder.decode([new_token]) for new_token in new_tokens])

In [ ]:
len(original_tokenizer) 

151665

In [ ]:
tokenizer.tokenize("vida")

['Ġvida']

In [9]:
tokenizer.tokenize('emprego')

['e', 'mpre', 'go']

In [10]:
original_tokenizer.tokenize('emprego')

['emp', 'rego']

In [12]:
original_tokenizer.tokenize('contrato')

['contr', 'ato']

In [33]:
tokenizer.encode(' contra')

[31050]

2. Update the model Vocabulary

In [5]:
# ----------------------------------------------------
#               Update Model Vocabulary               
# ----------------------------------------------------
# Save the original tokenizations
original_tokenization = {t: tokenizer.encode(t) for t in new_tokens}    # Necessary for the training bellow
tokenizer.add_tokens(list(new_tokens))
model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(156705, 1536)

3. Initialize the embeddings using the Weighted average $w_i = w_{i+1} \times K$ with $K=5$

In [8]:
# Step 4. Calculate the new embeddings for the new tokens
embed = model.get_input_embeddings().weight.clone().to('cpu')
new_embed = model.get_input_embeddings()

# Initialize the embedding using the weighted average model
K = 1.5 # Tested for values [1, 2, 3, 4, 5, 0.9, 0.8, 1.1, 1.2, ..., 1.6] and the best was 1.5 with (3.13%)
with torch.no_grad():
    for new_token in tqdm.tqdm(new_tokens, desc='Initializing the embeddings for the new_tokens'):
        new_token_id = tokenizer.encode(new_token)[0]
        # Find the old embedding for the token
        tokenization = original_tokenization[new_token]
        token_embed = torch.stack([embed[t_id] for t_id in tokenization]).to(DEVICE)
        # Calculating the embedding weights
        embedding_weights = torch.asarray([K**i if K**i < 2**64 else 0 for i in range(token_embed.shape[0], 0, -1)]).to(DEVICE)
        # embedding_weights = torch.asarray([K**i for i in range(token_embed.shape[0], 0, -1)]).to(DEVICE)
        embedding_weights = embedding_weights / embedding_weights.sum()

        # Create a new token embed using the weighted average of the embeddings
        new_token_embed = torch.sum(token_embed * embedding_weights[:, None], dim=0)
        # new_token_embed = token_embed[0]
        # Update embedding of the new_token in the hacked_model
        _ = new_embed.weight[new_token_id].data.copy_(new_token_embed)



Initializing the embeddings for the new_tokens: 100%|██████████| 5040/5040 [00:00<00:00, 8580.08it/s]


### Benchmark Computation

Calculating the results with the added tokens.

In [9]:
# Update model name
model.name_or_path = f'{MODEL}-ADDED_TOKENS_INIT_K={K}'

# Update the "max_new_tokens" (since we added new_tokens, we may have a different value than previously)
benchmark.config['max_new_tokens'] = max(len(tokenizer.encode(x)) for x in CalamePT().df['last_word'].values) + 1   # Maximum tokenization of predicted words
benchmark_results_new_tokens = benchmark.run(model, tokenizer, original_tokenizer)
print(f"`CalamePT` Accuracy for Baseline  Model `{model.name_or_path}` = {benchmark_results_new_tokens['CalamePT']['result']:.2%}")

<Qwen/Qwen2.5-1.5B-Instruct-ADDED_TOKENS_INIT_K=1.5> Calculating inferences for inputs: 100%|██████████| 260/260 [01:27<00:00,  2.98it/s]

`CalamePT` Accuracy for Baseline  Model `Qwen/Qwen2.5-1.5B-Instruct-ADDED_TOKENS_INIT_K=1.5` = 44.85%


In [10]:
phrase = ' Ela correu durante horas para alcançar a linha de'
max_new_tokens = 10
generation = tokenizer.decode(utils.generate(
    model, tokenizer,
    phrase,
    DEVICE, 
    False, False,
    max_new_tokens=max_new_tokens
)[0])
print(f'Generation with new tokenizer = `{generation}`')


input_ids = tokenizer.encode(phrase)
input_ids = [token for x in input_ids for token in original_tokenization.get(tokenizer.convert_ids_to_tokens(x), [x])]

for n in range(10):
    generation = model.generate(torch.Tensor([input_ids]).long().to(DEVICE), max_new_tokens=1)
    generated_id = generation[0, -1].item()
    input_ids.extend(
        [token for token in original_tokenization.get(tokenizer.convert_ids_to_tokens(generated_id), [generated_id])]
    )
print(f'Generation with "old tokenizer" = `{tokenizer.decode(input_ids)}`')

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generation with new tokenizer = ` Ela correu durante horas para alcançar a linha de meta, pero se quedó sin energía y per`
Generation with "old tokenizer" = ` Ela correu durante horas para alcançar a linha de chegada. 
- Foi um desaf`


## Model with additional Tokens + "Training"

Training the model (already has the added tokens)

In [ ]:
# ----------------------------------------------------
#     Update weights of Embeddings of new_tokens      
# ----------------------------------------------------
# Step 4.2 Using the training phrases to update the embedding weights
learning_rate = 1e-6
training_phrases: list[str] = CalamePT().prediction_prompts.to_list() # CalamePT dataset 
for new_token in tqdm.tqdm(new_tokens, desc='Updating the embeddings for the new tokens'):
    new_token_id = tokenizer.convert_tokens_to_ids(new_token)
    new_token = tokenizer.decode(new_token_id)
    phrases_to_generate_new_token = [p for phrase in training_phrases for p in phrase.split(new_token)[:-1] if new_token in phrase and len(p) > 0]

    if len(phrases_to_generate_new_token) == 0: continue
    # Creating the Batched dataset (to run generation for multiple phrases at the same time)
    dataloader = DataLoader(
        TextDataset(phrases_to_generate_new_token, tokenizer, max_length=max(len(tokenizer.tokenize(x)) for x in phrases_to_generate_new_token)),
        batch_size=GENERATION_BATCH_SIZE,
        shuffle=False
    )
    # Process the batches
    for batch in tqdm.tqdm(dataloader,  desc=f'  Generating tokens for new_token=`{new_token}` ', leave=False):
        # Move batch tensors to the correct device
        input_ids = batch['input_ids'].squeeze(1).to(DEVICE)
        attention_mask = batch['attention_mask'].squeeze(1).to(DEVICE)

        # Generate text
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=1,
            num_beams=1,
            num_return_sequences=1,
            return_dict_in_generate=True,
            output_logits=True,
            output_scores=True,
            output_hidden_states=True,
            pad_token_id=tokenizer.pad_token_id
        )

        # Extract the generated sequences and their scores
        generated_sequences = outputs.sequences
        predicted_logits = outputs.logits

        # Decode the input and generated sequences
        input_texts = tokenizer.batch_decode(input_ids, skip_special_tokens=True)
        generated_texts = tokenizer.batch_decode(generated_sequences, skip_special_tokens=True)
        with torch.no_grad():
            for i in range(len(input_texts)):
                logits = predicted_logits[0][i]
                logit_gradient = logits.max() - logits[new_token_id]
                embed_out = outputs.hidden_states[0][-1][i][-1]
                # normalize embed_out
                embed_out = embed_out / embed_out.norm()

                embed_in = new_embed.weight[new_token_id]

                # Update the embedding table
                _ = new_embed.weight[new_token_id].data.copy_((embed_in + logit_gradient * embed_out * learning_rate).to(DEVICE))

Updating the embeddings for the new tokens: 100%|██████████| 5040/5040 [40:42<00:00,  2.06it/s]  


### Update model name

Changing the model name to include the "added_tokens" to distinguish between the older model

In [12]:
model.name_or_path = f'{model.name_or_path} [TRAINED]'

### Run the benchmark

In [13]:
benchmark_results_new_tokens_trained = benchmark.run(model, tokenizer, original_tokenizer)
print(f"`CalamePT` Accuracy for Baseline Model `{model.name_or_path}` = {benchmark_results_new_tokens_trained['CalamePT']['result']:.2%}")

<Qwen/Qwen2.5-1.5B-Instruct-ADDED_TOKENS_INIT_K=1.5 [TRAINED]> Calculating inferences for inputs: 100%|██████████| 260/260 [01:41<00:00,  2.55it/s]

`CalamePT` Accuracy for Baseline Model `Qwen/Qwen2.5-1.5B-Instruct-ADDED_TOKENS_INIT_K=1.5 [TRAINED]` = 47.01%


SAVE ALL Benchmarks in a JSON file

In [15]:
import json
with open('/home/yali/MEGA/Hack The Tockenizer/tests/qwen2.5-benchmark-results_V2_run2.json', 'w', encoding='utf-8') as f:
    json.dump(benchmark.get_results(), f, indent=2, ensure_ascii=False)


# Analysing the Results

In [1]:
import json
import pandas as pd

with open('/home/yali/MEGA/Hack The Tockenizer/tests/qwen2.5-benchmark-results_V2_run2.json', 'r') as f:
    results: dict[str, dict[str, dict]] = json.load(f)

data = []
for model_name in results.keys():
    for benchmark_name in results[model_name].keys():
        for epoch_n, result in enumerate(results[model_name][benchmark_name]['results']):
            result: dict
            data.append(result.copy())
            data[-1]['model_name'] = model_name
            data[-1]['epoch_number'] = epoch_n

            r = data[-1].pop('benchmark_predictions')
            data[-1]['original_text']   = [gen["text"]          for gen in r]
            data[-1]['expected_word']   = [gen["correct_word"]  for gen in r]
            data[-1]['predicted_word']  = [gen["prediction"]    for gen in r]

df = pd.DataFrame(data).explode(['original_text', 'expected_word', 'predicted_word'])
# Changing the model name to only 2 letters, the first specifies if the model has been initialized or not, and the second one specifies if it has been trained or not
df['model_name'] = df['model_name'].map({
    'Qwen/Qwen2.5-1.5B-Instruct':                                   'BB',   # Baseline,    Baseline  
    'Qwen/Qwen2.5-1.5B-Instruct [TRAINED]':                         'BT',   # Baseline,    Trained  (currently not existing)
    'Qwen/Qwen2.5-1.5B-Instruct-ADDED_TOKENS_INIT_K=1.5':           'IB',   # Initialized, Baseline
    'Qwen/Qwen2.5-1.5B-Instruct-ADDED_TOKENS_INIT_K=1.5 [TRAINED]': 'IT',   # Initialized, Trained
})


df = df.pivot(
    index='original_text', columns=['model_name'], values=['predicted_word', 'expected_word']
)
df.columns = ['_'.join(col) for col in df.columns]
df = df[['predicted_word_BB', 'predicted_word_IB', 'predicted_word_IT', 'expected_word_BB']].reset_index().rename(columns={'expected_word_BB': 'expected_word'})

for model in [f'{init}{train}' for init in ['B', 'I'] for train in ['B', 'T']]:
    # There is no "BaselineTrained" model, YET
    if model == 'BT':
        continue
    df[f'correct_prediction_{model}'] = df[f'predicted_word_{model}'] == df['expected_word']

df

,original_text,predicted_word_BB,predicted_word_IB,predicted_word_IT,expected_word,correct_prediction_BB,correct_prediction_IB,correct_prediction_IT
0,A tragédia atingiu a família quando ele caiu ...,socorros,socorros,socorros,socorros,True,True,True
1,"A vida na aldeia é serena e simples, um refúg...",das,pequena,das,aldeia,False,False,False
2,Ela correu durante horas para alcançar a linh...,chegada,chegada,chegada,chegada,True,True,True
3,"""A vida é feita de novos desafios. Assumirei e...",melhor,melhor,melhor,valor,False,False,False
4,"""Apesar das dificuldades que enfrento, sigo em...",caminho,caminho,caminho,caminho,True,True,True
...,...,...,...,...,...,...,...,...
2071,"Wanda dos Santos, uma atleta afrodescendente, ...",jovens,pessoas,jovens,crianças,False,False,False
2072,À noite tudo fica escuro mas a sorte é que nos...,12,1228,87,estrelas,False,False,False
2073,É importante sempre confirmar as informações p...,atentos,sempre,atentos,atentos,True,False,True
2074,É na França que ocorre uma das competições de ...,atletas,atletas,atletas,ciclistas,False,False,False


Saving to csv file

In [17]:
df.to_csv('/home/yali/MEGA/Hack The Tockenizer/tests/qwen2.5-benchmark-results_V2_analysis.csv', index=False)

In [ ]:
# Next STEP: TRAIN the baseline model with a "normal" approach for the same amount of time and compare the results.